In [ ]:
import pandas as pd
import os
DATAPATH = "../data"
FIGUREPATH = "../figures"

#clean original dataset removing duplicates and "removed" drugs
df = pd.read_csv(os.path.join(DATAPATH, "eml_original.tsv"), sep="\t")
print(len(df))
df.drop_duplicates(subset="Medicine name", keep="first", inplace=True)
df=df[df["Status"]!= "Removed"]
print(len(df))

In [ ]:
drugs = df["Medicine name"].tolist()
for i,d in enumerate(drugs):
    if "isoniazid + pyridoxine + sulfamethoxazole + trimethoprim" in d:
        print(d)

In [ ]:
drugs = df["Medicine name"].tolist()
drugs_ind = []
idx=[]
for i,d in enumerate(drugs):
    if " + " in d:
        idx += [i]
        d1 = d.split(" +")[0]
        d2 = d.split("+ ")[1]
        drugs_ind += [d1]
        if "+" in d2:
            d3 = d2.split(" +")[0]
            d4 = d2.split("+ ")[1]
            drugs_ind += [d3]
            if "+" in d4:
                d5 = d4.split(" +")[0]
                d6 = d4.split("+ ")[1]
                drugs_ind += [d5]
                drugs_ind += [d6]
            else:
                drugs_ind += [d4]
        else:
            drugs_ind += [d2]

drugs = [d for j,d in enumerate(drugs)if j not in set(idx)]
drugs = set(drugs + [d for d in drugs_ind])
drugs = sorted((drugs))

In [ ]:
from ersilia.utils.identifiers.compound import CompoundIdentifier #requires Ersilia installed

smiles = []
for x in drugs:
    print(x)
    s=CompoundIdentifier.chemical_identifier_resolver(x)
    if s is None:
        print("no smiles found")
        smiles += ["NaN"]
    else:
        print(s)
        smiles += [s]

drug2smi = list(zip(drugs, smiles))
df = pd.DataFrame(drug2smi, columns=["drugs", "smiles"])

#save list and curate by hand missing smiles
df.to_csv(os.path.join(DATAPATH, "eml_list.csv"),index=False)

In [ ]:
from rdkit import Chem

df = pd.read_csv(os.path.join(DATAPATH, "eml_curated.csv"))

invalid_idx = []
can_s = []
for i,s in enumerate(df["smiles"].tolist()):
    print(s)
    try:
        mol = Chem.MolFromSmiles(s)
    except:
        mol=None    
    print(mol)
    if mol is None:
        invalid_idx += [i]
    else:
        cs = Chem.MolToSmiles(mol)
        can_s += [cs]

In [ ]:
df.drop(index=invalid_idx, axis=0, inplace=True)
df["can_smiles"] = can_s
df.to_csv(os.path.join(DATAPATH, "eml_canonical.csv"), index=False)